输入：单字形式的PE访存需求

[block_row_id: 0, PE_id: 0], [vector_addr: False, vector_len: 0][ (0, 1, 1), (2, 5, 203), (2, 14, 204), (4, 6, 400), (4, 11, 401), (5, 8, 515), (8, 13, 831), (9, 4, 937), (9, 9, 938), (10, 10, 1066), (10, 12, 1067), (11, 13, 1177), (12, 0, 1282), (12, 3, 1283), (13, 5, 1393), (14, 2, 1491), (14, 3, 1492), (14, 5, 1493), (14, 7, 1494), (15, 7, 1602), (15, 15, 1603) ]


输出：PE阵列每一行的访存需求范围，如果没有出现这行，则start=-1, end=-1
[block_row_id: 0, PE_id: 0], [vector_addr: False, vector_len: 0][(第0行, start, end), ... ,(第n行, start, end)]

In [63]:
import pandas as pd
import numpy as np
import re

# 读取TXT文件
txt_file = '../output/TEST1_A_access.txt'  
with open(txt_file, 'r') as file:
    lines = file.readlines()
# print(data)


# 定义一个函数来提取每行最后一个方括号内的内容，并将每个 (x, y, z) 作为一个元素提取
def extract_last_brackets(text):
    matches = re.findall(r'\[(.*?)\]', text)
    if matches:
        last_match = matches[-1]
        tuples = re.findall(r'\((\d+),\s*(\d+),\s*(\d+)\)', last_match)
        return [tuple(map(int, t)) for t in tuples]
    return []

# 提取每行最后一个方括号内的内容
all_extracted_data = []
for line in lines:
    extracted_data = extract_last_brackets(line)
    if extracted_data:
        all_extracted_data.append(extracted_data)

# 打印提取后的数据
# for i, data in enumerate(all_extracted_data):
#     print(f"PE {i}:{data}")

# with open('extracted_output.txt', 'w') as output_file:
#     for i, data in enumerate(all_extracted_data):
#         output_file.write(f"PE {i}: {data}\n")

################# 提取边界 ###################
# PEid的个数
tile_num = len(all_extracted_data)

# # 创建一个 n x 16 的矩阵，矩阵中的每个元素是 (0, 0)
interval_matrix = [[(0, 0) for _ in range(16)] for _ in range(tile_num)]

# # 将其转换为 numpy 数组
interval_matrix = np.array(interval_matrix)
# interval_matrix
for i, data_np in enumerate(all_extracted_data):
    # print(data_np)
    for (row, col, addr) in data_np:
        # print(row, col, addr)
        if interval_matrix[i][row][0] == 0:
            interval_matrix[i][row][0] = addr
            interval_matrix[i][row][1] = addr
            continue
        if addr > interval_matrix[i][row][1]:
            interval_matrix[i][row][1] = addr
            
with open('../output/TEST1_A_access_interval.txt', mode='w') as file:
    for i, row in enumerate(interval_matrix):
        formatted_row = [f"({x[0]}, {x[1]})" for x in row]
        row_str = ', '.join(formatted_row)
        file.write(f"PE {i}, [{row_str}]\n")